In [18]:
import sys
sys.path.append('..')
import importlib
import datagit.drift_evaluators_dry_run
importlib.reload(datagit.drift_evaluators_dry_run)
from datagit.drift_evaluators_dry_run import run_drift_evaluator
from datagit.drift_evaluators import default_drift_evaluator
from dotenv import load_dotenv
from github import Github
import os

load_dotenv("../.env")
gh_token = os.getenv("GH_TOKEN")
gh_client = Github(gh_token)
repo = os.getenv("REPO") or "gh_org/repo"

drift = run_drift_evaluator(gh_client=gh_client, repo_name=repo, commit_sha="b652e7a591cdf621eb9d13933a758293301316eb", drift_evaluator=default_drift_evaluator)
drift

{'should_alert': True,
 'message': 'Drift detected:\n- ~~🆕 0 addition~~\n- ♻️ 6 modifications\n- ~~🗑️ 0 deletion~~'}

In [ ]:
def example_drift_evaluator(data_drift_context):
    print(data_drift_context["reported_dataframe"])
    print( data_drift_context["computed_dataframe"])
    
    return {"should_alert": True, "message": "coucou"}

drift = run_drift_evaluator(gh_client=gh_client, repo_name=repo, commit_sha="b652e7a591cdf621eb9d13933a758293301316eb", drift_evaluator=default_drift_evaluator)
drift